<a href="https://colab.research.google.com/github/yiit8/object-detection-/blob/main/Car_Object_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yigitgun","key":"49c2c5fd9b39e87cdeb94cc886d4ec78"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download "sshikamaru/car-object-detection"

 94% 105M/112M [00:01<00:00, 75.3MB/s]
100% 112M/112M [00:01<00:00, 95.1MB/s]


In [ ]:
!unzip "/content/car-object-detection.zip" -d "/content/car-object-detection"

Archive:  /content/car-object-detection.zip
  inflating: /content/car-object-detection/data/sample_submission.csv  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25100.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25120.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25140.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25160.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25180.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25200.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25220.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25240.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_25260.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_26320.jpg  
  inflating: /content/car-object-detection/data/testing_images/vid_5_26400

In [ ]:
import os
import numpy as np
import torch
from PIL import Image


class CarDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "training_images"))))
       

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "training_images", self.imgs[idx])
        
        img = Image.open(img_path).convert("RGB")
 


        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
      

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)